In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, explode, split, regexp_replace, collect_list

from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Graph processing I')
    .config("spark.jars.packages", "graphframes:graphframes:0.5.0-spark2.1-s_2.11")
    .getOrCreate()
)

In [ ]:
from graphframes import *

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-2]) 

answers_input_path = os.path.join(project_path, 'data/answers')

users_input_path = os.path.join(project_path, 'data/users')

In [ ]:
answersDF = (
    spark
    .read
    .option('path', answers_input_path)
    .load()
).sample(0.1, False, 24).cache()

usersDF = (
    spark
    .read
    .option('path', users_input_path)
    .load()
)

In [ ]:
usersDF.dropDuplicates(['user_id']).count()

In [ ]:
answersDF.show(truncate=8, n=5)

In [ ]:
verticesDF = (
    answersDF
    .filter(col('user_id').isNotNull())
    .select('user_id')
    .distinct()
    .join(usersDF, 'user_id')
    .select(
        col('user_id').alias('id'),
        col('display_name')
    )
)

In [ ]:
edgesDF = (
    answersDF.alias('src')
    .join(
        answersDF.alias('dst'),
        'question_id'
    )
    .select(
        col('src.user_id').alias('src'), 
        col('dst.user_id').alias('dst')
    )
    .filter(col('src') != col('dst'))
)

In [ ]:
edgesDF.count()

In [ ]:
verticesDF.count()

In [ ]:
usersGraph = GraphFrame(verticesDF, edgesDF)

In [ ]:
communities = usersGraph.labelPropagation(5)

In [ ]:
(
  communities
    .groupBy('label')
    .agg(count('*').alias('cnt'))
    .orderBy(desc('cnt'))
).show()

In [ ]:
(
  communities
  .filter(col("label") == 1492)
).show()

In [ ]:
pr = (
   usersGraph.pageRank(tol=0.01)
 )

In [ ]:
(
  pr
  .vertices
  .orderBy(desc('pagerank'))
).show()